In [ ]:
import torch
import time
import numpy as np

import torch
import torchhd
import time

import torch
import torchhd
import time

import torch
import torchhd.functional as hd
import time

import torch
import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

TOTAL_CORES = 32

def hard_sign(x):
    return torch.where(x >= 0, 1.0, -1.0)

def torchhd_maptensor_sim(X, B, J, num_threads=1):
    torch.set_num_threads(num_threads)
    N, F = X.shape
    F_b, D = B.shape
    D_j, K = J.shape
    assert F == F_b and D == D_j
    y_pred = torch.empty(N, dtype=torch.long)
    start = time.perf_counter()
    H_raw = X @ B  # batch matmul [N, D]
    for i in range(N):
        h = hard_sign(H_raw[i])
        scores = h @ J           # dot similarity
        y_pred[i] = torch.argmax(scores)
    end = time.perf_counter()
    elapsed_ms = (end - start) * 1000
    return y_pred, elapsed_ms

def torchhd_maptensor_sim_low_batches(X, B, J, num_threads=1):
    torch.set_num_threads((num_threads//2) if ((num_threads//2) != TOTAL_CORES) else num_threads)
    N, F = X.shape
    F_b, D = B.shape
    D_j, K = J.shape
    assert F == F_b and D == D_j
    y_pred = torch.empty(N, dtype=torch.long)
    scores_full = torch.empty((N, K), dtype=torch.long)
    start = time.perf_counter()
    H_raw = X @ B 
    H = torch.empty_like(H_raw)
    for i in range(N):
        H[i] = hard_sign(H_raw[i])
    for i in range(N):
        scores_full[i] = H[i] @ J 
    for i in range(N): 
        y_pred[i] = torch.argmax(scores_full[i])
    end = time.perf_counter()
    elapsed_ms = (end - start) * 1000
    return y_pred, elapsed_ms

def run_torchhd_maptensor_sim(N, F, D, K, threads=4, runs=10):
    torch.manual_seed(42)
    X = torch.randn(N, F, dtype=torch.float32)
    B = torch.randn(F, D, dtype=torch.float32)
    M = torch.randn(K, D, dtype=torch.float32)
    J = M.T 
    latencies = []

    if N <= 512:
        torchhd_func = torchhd_maptensor_sim_low_batches
    else:
        torchhd_func = torchhd_maptensor_sim

    for _ in range(runs):
        _, elapsed = torchhd_func(X, B, J, num_threads=threads)
        latencies.append(elapsed)
    avg_latency = sum(latencies) / len(latencies)
    latency_per_image = avg_latency / N
    throughput = 1000.0 / latency_per_image
    '''
    print(f"--- TorchHD MAPTensor Simulation over {runs} runs ---")
    print(f"Samples: {N}, Threads: {threads}")
    print(f"Avg Latency: {avg_latency:.2f} ms")
    print(f"Latency per Image: {latency_per_image:.4f} ms")
    print(f"Throughput: {throughput:.2f} images/sec")
    '''
    return avg_latency, latency_per_image, throughput

def torchhd_get_processed_dict(F, D, K, runs):
    torchhd_HT_dict = {}
    for N in [1024, 2048, 4096, 8192, 16384, 32768]:
        lat_norm_list, thru_list, lat_end_list, T_list = [], [], [], []
        for T in [2, 4, 8, 16, 32, 64]:
            loc_lat_end, loc_lat_norm, loc_thru = run_torchhd_maptensor_sim(N, F, D, K, T, runs=runs)
            lat_norm_list.append(loc_lat_norm)
            thru_list.append(loc_thru)
            lat_end_list.append(loc_lat_end)
            T_list.append(T)
        torchhd_HT_dict[N] = {'latency_norm': lat_norm_list,
                              'throughput': thru_list,
                              'latency_end': lat_end_list,
                              'T': [t // 2 for t in T_list],
                              '2T': T_list}   
    torchhd_LL_dict = {}
    for N in [32, 64, 128, 256, 512, 1024, 2048]:
        lat_norm_list, thru_list, lat_end_list, T_list = [], [], [], []
        for T in [2, 4, 8, 16, 32, 64]:
            loc_lat_end, loc_lat_norm, loc_thru = run_torchhd_maptensor_sim(N, F, D, K, T, runs=runs)
            lat_norm_list.append(loc_lat_norm)
            thru_list.append(loc_thru)
            lat_end_list.append(loc_lat_end)
            T_list.append(T)
        torchhd_LL_dict[N] = {'latency_norm': lat_norm_list,
                              'throughput': thru_list,
                              'latency_end': lat_end_list,
                              'T': [t // 2 for t in T_list],
                              '2T': T_list}
    return (torchhd_HT_dict, torchhd_LL_dict)

def get_speedup_versus(result, baseline):
    speedup_dict = {}
    for N in result:
        my_thru = result[N]['throughput']
        base_thru = baseline[N]['throughput']
        speedups = [x/y for x, y in zip(my_thru, base_thru)]
        speedup_dict[N] = {'speedup': speedups, 'cores': result[N]['2T']}
    return speedup_dict

def get_speedup_itself(result):
    speedup_dict = {}
    for N in result:
        for key in result[N]:
            baseline = result[N][key]['throughput'][0]
            speedups = [throughput/baseline for throughput in result[N][key]['throughput']]
            cores = result[N][key]['2T']
            if N not in speedup_dict:
                speedup_dict[N] = {}
            speedup_dict[N][key] = {'speedup': speedups, 'cores': cores}
    return speedup_dict

In [ ]:
def process_dump(dump_dict):
    
    D, F, K = dump_dict["D"][0], dump_dict["F"][0], dump_dict["K"][0]
    dataset, variant = dump_dict["dataset"], dump_dict["variant"]
    
    R_list = dump_dict["R"]
    T_list = dump_dict["T"]
    N_list = dump_dict["N"]
    nfdk_list = dump_dict["n"]

    
    lat_norm_list = dump_dict["latency_ms_per_image"]
    thru_norm_list = dump_dict["throughput_img_per_sec"]
    end_lat_list = dump_dict["total_runtime_ms"]

    metadata_dict = {'D': D, 'F': F, 'K': K, 'dataset': dataset, 'variant': variant}
    out_dict = {}

    for element in zip(N_list, nfdk_list, R_list, T_list, lat_norm_list, thru_norm_list, end_lat_list):
        _N, _nfdk, _R, _T, _lat_norm, _thru_norm, _end_lat = element
        if not _N in out_dict:
            out_dict[_N] = {}
        key = str(_nfdk) + str("_") + str(_R)
        if not key in out_dict[_N]:
            out_dict[_N][key] =                               {'latency_norm': [],      # per image, in ms
                                                               'throughput': [],        # images per sec
                                                               'latency_end': [],       # end to end, in ms
                                                               'T': [],
                                                               '2T': []}
        out_dict[_N][key]['latency_norm'].append(_lat_norm)
        out_dict[_N][key]['throughput'].append(_thru_norm)
        out_dict[_N][key]['latency_end'].append(_end_lat)
        out_dict[_N][key]['T'].append(_T)
        out_dict[_N][key]['2T'].append(2*_T)
    
    return (out_dict, metadata_dict)

def get_max_thru(processed_dict):
    max_dict = {}
    for N in processed_dict:
        all_dicts = processed_dict[N]        
        lat_norm_MIN, thru_MAX, lat_end_MIN, T = [], [], [], []
        for key in all_dicts:
            length = len(all_dicts[key]["T"])
            break
        for i in range(length):
            loccand_lat_norm, loccand_thru, loccand_lat_end, loccand_T = [],  [],  [],  []
            for key in all_dicts:
                loccand_lat_norm.append(all_dicts[key]['latency_norm'][i])
                loccand_thru.append(all_dicts[key]['throughput'][i])
                loccand_lat_end.append(all_dicts[key]['latency_end'][i])
                loccand_T.append(all_dicts[key]['T'][i])
            assert len(set(loccand_T)) == 1
            T.append(loccand_T[0])
            lat_norm_MIN.append(min(loccand_lat_norm))
            thru_MAX.append(max(loccand_thru))
            lat_end_MIN.append(min(loccand_lat_end))
        max_dict[N] = {'latency_norm': lat_norm_MIN,
                       'throughput': thru_MAX,
                       'latency_end': lat_end_MIN,
                       'T': T,
                       '2T': [2*k for k in T]}
    return max_dict   

In [ ]:
HT_dump = \
{
"D": [10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000],
"F": [784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784],
"K": [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
"N": [1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,8192,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,16384,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768,32768],
"R": [8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16],
"T": [1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32],
"binding": "binding_used",
"d": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
"dataset": "MNIST",
"datetime": "Sun May 11 22:20:12 2025",
"f": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
"k": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
"latency_ms_per_image": [0.4775749146938324,0.4572850465774536,0.2450718879699707,0.2391040325164795,0.2148730456829071,0.20481228828430176,0.1141757145524025,0.11314990371465683,0.12138445675373077,0.09230311214923859,0.13683578372001648,0.08441150933504105,0.30412399768829346,0.29537028074264526,0.15426115691661835,0.1558055728673935,0.08217223733663559,0.07837719470262527,0.06371042877435684,0.06489329040050507,0.04425695165991783,0.04030345752835274,0.055926475673913956,0.04241596534848213,0.4863435924053192,0.46229350566864014,0.24272504448890686,0.23241786658763885,0.14494717121124268,0.13297688961029053,0.11726554483175278,0.10177528113126755,0.13567160069942474,0.10421550273895264,0.11609062552452087,0.07288476824760437,0.29572898149490356,0.2873936891555786,0.15261200070381165,0.1489119529724121,0.08370383083820343,0.07861718535423279,0.06385171413421631,0.06034434959292412,0.041723813861608505,0.0403614342212677,0.042799677699804306,0.04022728279232979,0.48679986596107483,0.46479856967926025,0.24433894455432892,0.23690852522850037,0.19797828793525696,0.19062860310077667,0.11449794471263885,0.11059414595365524,0.10561324656009674,0.08193108439445496,0.1034117266535759,0.06855751574039459,0.2951068878173828,0.29407137632369995,0.14892245829105377,0.15307144820690155,0.07698679715394974,0.07568444311618805,0.044440850615501404,0.043848514556884766,0.041319094598293304,0.03772644326090813,0.03589179739356041,0.03462761268019676,0.5061726570129395,0.4794563055038452,0.25428810715675354,0.2511560618877411,0.1307348906993866,0.17375098168849945,0.11911078542470932,0.11778756976127625,0.11197999119758606,0.08039239048957825,0.1118454560637474,0.06881348788738251,0.47245895862579346,0.4495548903942108,0.23096008598804474,0.22554908692836761,0.1172521561384201,0.11502687633037567,0.06946415454149246,0.06835135072469711,0.0435369648039341,0.040408216416835785,0.03675607591867447,0.033591825515031815,0.5549381971359253,0.4983423948287964,0.2715403735637665,0.25880420207977295,0.13634712994098663,0.12748397886753082,0.12878768146038055,0.12040271610021591,0.11586233973503113,0.09441603720188141,0.09929775446653366,0.0698557123541832,0.34735968708992004,0.3237764239311218,0.1691870242357254,0.16434071958065033,0.12159352749586105,0.11955846846103668,0.06750665605068207,0.066177137196064,0.043633099645376205,0.04205920547246933,0.03400522843003273,0.03309212997555733,0.5609405636787415,0.5111827254295349,0.28449445962905884,0.25884613394737244,0.16876220703125,0.13260173797607422,0.11559562385082245,0.12518885731697083,0.12169896066188812,0.09395736455917358,0.10566475987434387,0.07558812201023102,0.3360852301120758,0.3242116868495941,0.17480336129665375,0.1693582683801651,0.12633183598518372,0.08689597994089127,0.05882561579346657,0.062320370227098465,0.050865206867456436,0.04868548363447189,0.03848963975906372,0.03331589698791504],
"n": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
"throughput_img_per_sec": [2093.912353515625,2186.81982421875,4080.435546875,4182.27978515625,4653.9111328125,4882.51953125,8758.4296875,8837.833984375,8238.287109375,10833.87109375,7308.02978515625,11846.7255859375,3288.13232421875,3385.5810546875,6482.513671875,6418.25537109375,12169.560546875,12758.8125,15696.017578125,15409.9140625,22595.3203125,24811.767578125,17880.619140625,23576.02734375,2056.159423828125,2163.1279296875,4119.88818359375,4302.59521484375,6899.0654296875,7520.103515625,8527.654296875,9825.568359375,7370.73974609375,9595.501953125,8613.9599609375,13720.2880859375,3381.474609375,3479.547607421875,6552.56494140625,6715.3779296875,11946.8837890625,12719.865234375,15661.2861328125,16571.560546875,23967.126953125,24776.126953125,23364.662109375,24858.751953125,2054.232177734375,2151.4697265625,4092.67529296875,4221.03857421875,5051.05908203125,5245.80224609375,8733.78125,9042.0703125,9468.509765625,12205.3798828125,9670.0830078125,14586.29296875,3388.602783203125,3400.534912109375,6714.90380859375,6532.89697265625,12989.2412109375,13212.7548828125,22501.8203125,22805.7890625,24201.884765625,26506.607421875,27861.51953125,28878.685546875,1975.6103515625,2085.69580078125,3932.54736328125,3981.588134765625,7649.0673828125,5755.36328125,8395.544921875,8489.8603515625,8930.1669921875,12438.98828125,8940.908203125,14532.0341796875,2116.5859375,2224.42236328125,4329.75244140625,4433.62451171875,8528.6279296875,8693.62109375,14395.9150390625,14630.2880859375,22968.986328125,24747.44140625,27206.38671875,29769.146484375,1802.00244140625,2006.6524658203125,3682.69384765625,3863.9248046875,7334.220703125,7844.12353515625,7764.7177734375,8305.4599609375,8630.931640625,10591.4208984375,10070.720703125,14315.2216796875,2878.8603515625,3088.551025390625,5910.61865234375,6084.91943359375,8224.1220703125,8364.1083984375,14813.3544921875,15110.9599609375,22918.380859375,23776.009765625,29407.2421875,30218.666015625,1782.719970703125,1956.2476806640625,3515.0068359375,3863.299072265625,5925.4970703125,7541.37939453125,8650.8466796875,7987.931640625,8216.9970703125,10643.125,9463.8935546875,13229.591796875,2975.435791015625,3084.404541015625,5720.7138671875,5904.642578125,7915.66162109375,11508.01171875,16999.396484375,16046.1171875,19659.8046875,20540.00390625,25981.017578125,30015.69921875],
"total_runtime_ms": [489.0367126464844,468.2598876953125,250.95361328125,244.842529296875,220.02999877929688,209.727783203125,116.91593170166016,115.8655014038086,124.29768371582031,94.51838684082031,140.11984252929688,86.43738555908203,311.4229736328125,302.45916748046875,157.9634246826172,159.54490661621094,84.14437103271484,80.25824737548828,65.2394790649414,66.45072937011719,45.31911849975586,41.2707405090332,57.26871109008789,43.4339485168457,996.0316772460938,946.777099609375,497.10089111328125,475.9917907714844,296.851806640625,272.336669921875,240.1598358154297,208.43577575683594,277.8554382324219,213.433349609375,237.75360107421875,149.26800537109375,605.6529541015625,588.582275390625,312.54937744140625,304.9716796875,171.42544555664063,161.00799560546875,130.768310546875,123.5852279663086,85.45037078857422,82.66021728515625,87.65373992919922,82.3854751586914,1993.9322509765625,1903.81494140625,1000.8123168945313,970.3773193359375,810.9190673828125,780.8147583007813,468.98358154296875,452.9936218261719,432.59185791015625,335.5897216796875,423.5744323730469,280.81158447265625,1208.7578125,1204.516357421875,609.9863891601563,626.9806518554688,315.3379211425781,310.00347900390625,182.02972412109375,179.603515625,169.24301147460938,154.5275115966797,147.01280212402344,141.83470153808594,4146.56640625,3927.7060546875,2083.128173828125,2057.470458984375,1070.980224609375,1423.3680419921875,975.7555541992188,964.915771484375,917.340087890625,658.574462890625,916.2379760742188,563.7200927734375,3870.3837890625,3682.753662109375,1892.0250244140625,1847.6981201171875,960.5296630859375,942.3001708984375,569.0503540039063,559.9342651367188,356.6548156738281,331.02410888671875,301.10577392578125,275.1842346191406,9092.107421875,8164.841796875,4448.91748046875,4240.248046875,2233.911376953125,2088.697509765625,2110.057373046875,1972.6781005859375,1898.28857421875,1546.912353515625,1626.8944091796875,1144.5159912109375,5691.14111328125,5304.7529296875,2771.960205078125,2692.558349609375,1992.1883544921875,1958.845947265625,1106.029052734375,1084.2462158203125,714.8847045898438,689.0980224609375,557.1416625976563,542.1814575195313,18380.900390625,16750.435546875,9322.314453125,8481.8701171875,5530.0,4345.09375,3787.83740234375,4102.1884765625,3987.83154296875,3078.794921875,3462.4228515625,2476.87158203125,11012.8408203125,10623.7685546875,5727.95654296875,5549.53173828125,4139.6416015625,2847.407470703125,1927.5977783203125,2042.1138916015625,1666.7510986328125,1595.325927734375,1261.228515625,1091.6953125],
"variant": "high_throughput"
}
LL_dump = \
{
  "D": [10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000],
  "F": [784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784],
  "K": [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
  "N": [32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,128,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,256,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048],
  "R": [8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16,8,16],
  "T": [1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32,1,1,2,2,4,4,8,8,16,16,32,32],
  "binding": "binding_used",
  "d": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
  "dataset": "MNIST",
  "datetime": "Tue Jun  3 22:22:57 2025",
  "f": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
  "k": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
  "latency_ms_per_image": [1.1251634359359741,1.0225846767425537,0.7470424771308899,0.6870083212852478,0.5710524916648865,0.49071913957595825,0.5620571374893188,0.48947298526763916,0.6105679273605347,0.42969873547554016,0.6135158538818359,0.6409336924552917,0.9409196972846985,0.9000160098075867,0.6365660429000854,0.7456048727035522,0.4744907319545746,0.46957072615623474,0.42041176557540894,0.42453011870384216,0.4760752022266388,0.4501372277736664,0.7386138439178467,0.8078811168670654,0.8999365568161011,0.8884832262992859,0.5422713756561279,0.5360515713691711,0.3745589554309845,0.3493880033493042,0.30699798464775085,0.2541700601577759,0.37733566761016846,0.2758665382862091,0.42219436168670654,0.3671364188194275,0.7197546362876892,0.7318941354751587,0.45721346139907837,0.4435766339302063,0.2980789840221405,0.29783716797828674,0.2689010202884674,0.2553234100341797,0.28217819333076477,0.21518462896347046,0.33597755432128906,0.35711732506752014,0.8113208413124084,0.8090313076972961,0.45223093032836914,0.44871529936790466,0.4161211848258972,0.39697766304016113,0.24077393114566803,0.202203631401062,0.19362932443618774,0.18588979542255402,0.22498184442520142,0.21282702684402466,0.6080576777458191,0.6031345129013062,0.34451717138290405,0.3547932505607605,0.22713549435138702,0.21824383735656738,0.15595698356628418,0.1538228988647461,0.13772064447402954,0.12496332079172134,0.20127956569194794,0.21584533154964447,0.7730161547660828,0.751735508441925,0.41586577892303467,0.4065645933151245,0.37308961153030396,0.36526545882225037,0.20852863788604736,0.20697054266929626,0.16138099133968353,0.12815481424331665,0.1594964563846588,0.11891456693410873,0.5788584351539612,0.5627402067184448,0.30605727434158325,0.29931938648223877,0.17222820222377777,0.17087768018245697,0.11399476230144501,0.11672671139240265,0.09867656230926514,0.08808796107769012,0.10231360048055649,0.16357463598251343,0.754280686378479,0.7262504696846008,0.39103594422340393,0.38335177302360535,0.3596922755241394,0.34097331762313843,0.20171642303466797,0.18953649699687958,0.14323627948760986,0.11559444665908813,0.11209461092948914,0.09279759973287582,0.5509102940559387,0.5378710031509399,0.28389236330986023,0.28429198265075684,0.1592448651790619,0.15234467387199402,0.09618718177080154,0.09589197486639023,0.07718756794929504,0.06954563409090042,0.08192212879657745,0.0808587297797203,0.7507007718086243,0.7195624113082886,0.37934544682502747,0.37010103464126587,0.19830213487148285,0.1934225857257843,0.1885857731103897,0.1825372874736786,0.1276729702949524,0.10258761048316956,0.1223490834236145,0.07817710936069489,0.5290778279304504,0.5192387104034424,0.26995038986206055,0.2664366066455841,0.1418352872133255,0.14115339517593384,0.08329135924577713,0.08280925452709198,0.06598280370235443,0.06395109742879868,0.06168149411678314,0.06379058957099915,1.2913771867752075,0.7521933317184448,0.37027621269226074,0.36050939559936523,0.1930844932794571,0.18782812356948853,0.13485965132713318,0.11365313827991486,0.14474385976791382,0.11160775274038315,0.09417225420475006,0.08658338338136673,0.5129278898239136,0.5087260007858276,0.25899821519851685,0.2570151686668396,0.1357826292514801,0.13442441821098328,0.08062326163053513,0.07801355421543121,0.06186293810606003,0.061495017260313034,0.04960933327674866,0.04955942556262016],
  "n": [16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32,16,16,16,16,16,16,16,16,16,16,16,16,32,32,32,32,32,32,32,32,32,32,32,32],
  "throughput_img_per_sec": [888.7598266601563,977.9141845703125,1338.6119384765625,1455.58642578125,1751.1524658203125,2037.8255615234375,1779.1785888671875,2043.013671875,1637.8193359375,2327.2119140625,1629.94970703125,1560.2237548828125,1062.7899169921875,1111.09130859375,1570.928955078125,1341.1929931640625,2107.522705078125,2129.6044921875,2378.620361328125,2355.545654296875,2100.508544921875,2221.544677734375,1353.887451171875,1237.805908203125,1111.189453125,1125.513671875,1844.0950927734375,1865.4923095703125,2669.806640625,2862.1474609375,3257.350341796875,3934.37353515625,2650.160400390625,3624.94140625,2368.577392578125,2723.783203125,1389.3624267578125,1366.31787109375,2187.162109375,2254.40185546875,3354.8154296875,3357.539306640625,3718.84033203125,3916.601318359375,3543.85986328125,4647.17236328125,2976.38916015625,2800.2001953125,1232.55810546875,1236.046142578125,2211.259521484375,2228.58447265625,2403.146240234375,2519.033447265625,4153.2734375,4945.50927734375,5164.5068359375,5379.53173828125,4444.80322265625,4698.6513671875,1644.580810546875,1658.0048828125,2902.61279296875,2818.54296875,4402.65869140625,4582.03076171875,6412.0244140625,6500.98291015625,7261.07568359375,8002.34814453125,4968.2138671875,4632.947265625,1293.634033203125,1330.255126953125,2404.6220703125,2459.6337890625,2680.3212890625,2737.735107421875,4795.50439453125,4831.60546875,6196.5166015625,7803.0625,6269.73193359375,8409.3984375,1727.5379638671875,1777.01904296875,3267.3623046875,3340.9130859375,5806.25,5852.13916015625,8772.3330078125,8567.01953125,10134.119140625,11352.2890625,9773.8720703125,6113.4169921875,1325.7664794921875,1376.9354248046875,2557.309814453125,2608.570068359375,2780.154052734375,2932.78076171875,4957.45458984375,5276.02880859375,6981.4716796875,8650.9345703125,8921.03515625,10776.140625,1815.1776123046875,1859.1817626953125,3522.461669921875,3517.51025390625,6279.63720703125,6564.0625,10396.3955078125,10428.4013671875,12955.4541015625,14379.046875,12206.7138671875,12367.248046875,1332.088623046875,1389.7335205078125,2636.119873046875,2701.965087890625,5042.81005859375,5170.02685546875,5302.626953125,5478.3330078125,7832.51171875,9747.765625,8173.33447265625,12791.46875,1890.0810546875,1925.896484375,3704.384521484375,3753.23779296875,7050.431640625,7084.49169921875,12006.046875,12075.9453125,15155.4638671875,15636.9482421875,16212.318359375,15676.29296875,774.3671264648438,1329.4454345703125,2700.6865234375,2773.852783203125,5179.080078125,5324.0166015625,7415.1162109375,8798.701171875,6908.75537109375,8959.951171875,10618.8388671875,11549.5595703125,1949.591796875,1965.694580078125,3861.030517578125,3890.821044921875,7364.7119140625,7439.1240234375,12403.3681640625,12818.28515625,16164.767578125,16261.4794921875,20157.49609375,20177.796875],
  "total_runtime_ms": [36.00522994995117,32.72270965576172,23.905359268188477,21.98426628112793,18.273679733276367,15.703012466430664,17.985828399658203,15.663135528564453,19.53817367553711,13.750359535217285,19.63250732421875,20.509878158569336,30.10943031311035,28.800512313842773,20.370113372802734,23.859355926513672,15.183703422546387,15.026263236999512,13.453176498413086,13.58496379852295,15.234406471252441,14.404391288757324,23.635643005371094,25.852195739746094,57.59593963623047,56.8629264831543,34.70536804199219,34.30730056762695,23.971773147583008,22.36083221435547,19.647871017456055,16.266883850097656,24.14948272705078,17.655458450317383,27.02043914794922,23.49673080444336,46.06429672241211,46.841224670410156,29.261661529541016,28.388904571533203,19.077054977416992,19.06157875061035,17.209665298461914,16.3406982421875,18.059404373168945,13.77181625366211,21.5025634765625,22.85550880432129,103.84906768798828,103.5560073852539,57.88555908203125,57.4355583190918,53.263511657714844,50.813140869140625,30.819063186645508,25.882064819335938,24.78455352783203,23.793893814086914,28.79767608642578,27.241859436035156,77.83138275146484,77.20121765136719,44.09819793701172,45.413536071777344,29.07334327697754,27.935211181640625,19.962493896484375,19.6893310546875,17.62824249267578,15.995305061340332,25.763784408569336,27.628202438354492,197.8921356201172,192.4442901611328,106.46163940429688,104.08053588867188,95.51094055175781,93.5079574584961,53.383331298828125,52.984458923339844,41.313533782958984,32.80763244628906,40.831092834472656,30.442129135131836,148.18775939941406,144.06149291992188,78.35066223144531,76.62576293945313,44.09041976928711,43.744686126708984,29.182659149169922,29.882038116455078,25.261199951171875,22.550518035888672,26.19228172302246,41.87510681152344,386.19171142578125,371.8402404785156,200.2104034423828,196.27610778808594,184.16244506835938,174.57833862304688,103.27880859375,97.04268646240234,73.33697509765625,59.184356689453125,57.39244079589844,47.51237106323242,282.0660705566406,275.38995361328125,145.35289001464844,145.5574951171875,81.53337097167969,78.00047302246094,49.24783706665039,49.0966911315918,39.52003479003906,35.607364654541016,41.944129943847656,41.3996696472168,768.7175903320313,736.8319091796875,388.4497375488281,378.98345947265625,203.06138610839844,198.06472778320313,193.11183166503906,186.91818237304688,130.73712158203125,105.04971313476563,125.28546142578125,80.05335998535156,541.7756958007813,531.700439453125,276.42919921875,272.8310852050781,145.2393341064453,144.54107666015625,85.29035186767578,84.79667663574219,67.56639099121094,65.48592376708984,63.16184997558594,65.32156372070313,2644.740478515625,1540.491943359375,758.32568359375,738.3232421875,395.4370422363281,384.6719970703125,276.19256591796875,232.76162719726563,296.4354248046875,228.5726776123047,192.86477661132813,177.32276916503906,1050.476318359375,1041.870849609375,530.4283447265625,526.3670654296875,278.08282470703125,275.30120849609375,165.11643981933594,159.77175903320313,126.69529724121094,125.9417953491211,101.59991455078125,101.4977035522461],
  "variant": "low_latency"
}

In [ ]:
HT_processed, HT_metadata = process_dump(HT_dump)
LL_processed, LL_metadata = process_dump(LL_dump)

HT_optimal = get_max_thru(HT_processed)
LL_optimal = get_max_thru(LL_processed)

baseline_HT, baseline_LL = torchhd_get_processed_dict(F=HT_metadata['F'],
                                                      D=HT_metadata['D'],
                                                      K=HT_metadata['K'],
                                                      runs=3)

print('HT_optimal: \n', HT_optimal)
print('baseline_HT: \n', baseline_HT)
print('LL_optimal: \n', LL_optimal)
print('baseline_LL: \n', baseline_LL)

In [ ]:
speedup_HT, speedup_LL = get_speedup_versus(result=HT_optimal, baseline=baseline_HT), get_speedup_versus(result=LL_optimal, baseline=baseline_LL)

print('speedup_HT: \n', speedup_HT)
print('speedup_LL: \n', speedup_LL)

In [ ]:
def plot_speedup_vs_cores(dataset_name, platform, speedup_LL, speedup_HT):
    """
    Plots throughput speedup vs number of cores for a given dataset and platform.
    
    Args:
        dataset_name (str): Name of the dataset (e.g., "MNIST")
        platform (str): Either "AMD" or "INTEL"
        speedup_LL (dict): Dictionary with batch sizes as keys for ScalableHD-S
        speedup_HT (dict): Dictionary with batch sizes as keys for ScalableHD-L
        save_path (str): Path to save the generated PDF plot
    """
    # Choose platform symbol
    if platform.upper() == "AMD":
        symbol = "★"
        max_cores = 64
    elif platform.upper() == "INTEL":
        symbol = "♠"
        max_cores = 128
    else:
        raise ValueError("Platform must be either 'AMD' or 'INTEL'")

    fig, ax = plt.subplots(figsize=(12, 7))

    # Colors
    S_colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(speedup_LL)))
    L_colors = plt.cm.Greens(np.linspace(0.3, 0.9, len(speedup_HT)))

    # Plot LL (S)
    for i, (N, data) in enumerate(speedup_LL.items()):
        ax.plot(data['cores'], data['speedup'], color=S_colors[i], marker='o', linestyle='-')

    # Plot HT (L)
    for i, (N, data) in enumerate(speedup_HT.items()):
        ax.plot(data['cores'], data['speedup'], color=L_colors[i], marker='^', linestyle='--')

    # Axes formatting
    ax.set_xscale('log', base=2)
    ax.set_xticks([2, 4, 8, 16, 32, 64] if max_cores == 64 else [2, 4, 8, 16, 32, 64, 128])
    ax.get_xaxis().set_major_formatter(plt.ScalarFormatter())
    ax.set_xlim(1.5, max_cores + 5)

    # Title and labels
    ax.set_title(f'{dataset_name} Throughput Speedup vs Number of Cores ({symbol})', fontsize=18, fontweight='bold')
    ax.set_xlabel('Number of Cores', fontsize=16, fontweight='bold')
    ax.set_ylabel('Throughput Speedup', fontsize=16, fontweight='bold')
    ax.tick_params(axis='both', which='major', labelsize=14, width=2)
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontweight('bold')

    # Legends
    logN_labels_LL = [fr'{int(np.log2(N))}' for N in speedup_LL.keys()]
    logN_labels_HT = [fr'{int(np.log2(N))}' for N in speedup_HT.keys()]
    logN_legend_handles_LL = [
        plt.Line2D([0], [0], color=color, marker='o', linestyle='-', label=lbl)
        for color, lbl in zip(S_colors, logN_labels_LL)
    ]
    logN_legend_handles_HT = [
        plt.Line2D([0], [0], color=color, marker='^', linestyle='--', label=lbl)
        for color, lbl in zip(L_colors, logN_labels_HT)
    ]
    variant_legend_handles = [
        Patch(facecolor='red', label='S'),
        Patch(facecolor='green', label='L')
    ]

    combined_handles = logN_legend_handles_LL + logN_legend_handles_HT + variant_legend_handles
    ax.legend(handles=combined_handles,
              title=r'$\mathrm{log}_2 N$',
              loc='upper left', bbox_to_anchor=(0.0, 1.00),
              fontsize=11, title_fontsize=13,
              ncol=1, borderpad=1.0, labelspacing=0.7)

    save_path = 'figs' + '/' + str('throughput') + str('_') + str(dataset_name) + str('_') + str(platform)

    plt.tight_layout()
    plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.show()

In [ ]:
plot_speedup_vs_cores('MNIST', 'AMD', speedup_LL, speedup_HT)